# Project Cypher  
## Assignment  

Chama is a relatively new, modern company, with an IOS App and Android App. To grow our company we must make a special effort in collecting all information available around the App and make it available for everyone in the company. Some information about App usage is generated by our App and our backend API. Other sources of information, like the Google Play Store, can provide very useful insights on App performance and user rating. Chama is very data-oriented and, most decisions are made based on appropriate metrics, therefore, data quality is a must and a concern of everyone involved.

Some event data come as json files and need some transformation to be structured as tables. Convert the case.json file to 3 csv files, using the programming language of your choice, with the following rules:

**CuratedOfferOptions.csv:**
```
CurationProvider: in quotes
OfferId: in quotes
DealerId: in quotes
UniqueOptionId: in quotes
OptionId: in quotes
IsMobileDealer: without quotes
IsOpen: without quotes
Eta: in quotes
ChamaScore: without quotes
ProductBrand: in quotes
IsWinner: without quotes
MinimumPrice: without quotes
MaximumPrice: without quotes
DynamicPrice: without quotes
FinalPrice: without quotes
DefeatPrimaryReason: in quotes
DefeatReasons: in quotes
EnqueuedTimeSP: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)
```


**DynamicPriceOption.csv:**
```
Provider: in quotes
OfferId: in quotes
UniqueOptionId: in quotes
BestPrice: without quotes
EnqueuedTimeSP: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)
```


**DynamicPriceRange.csv:**
```
Provider: in quotes
OfferId: in quotes
MinGlobal: without quotes
MinRecommended: without quotes
MaxRecommended: without quotes
DifferenceMinRecommendMinTheory: without quotes
EnqueuedTimeSP: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)
```


It is important to note the stipulation of which cols should be strs and which should be ints.

## Setup

In [1]:
#############
# Setup
#############

import pandas as pd
import numpy as np
import json

df = pd.read_json(r"C:\Users\brice\Downloads\company_data\datasets\case.json", orient='columns')

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_colwidth', 5000)


## Explore

In [2]:
#############
# Explore
#############

def missing_data(df):
    """given df, print number of missing values per col and percent of missing values per col
    sort: desc"""
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Pecent'])
    print("\n\nMissing Data: \n",missing_data)

    
    
print("Raw Data Head:\n\n", df.head(5), "\n", sep="")
print("Value Counts for EventName(col): \n\n", df['EventName'].value_counts(), sep="")
missing_data(df)

Raw Data Head:

           EnqueuedTimeUtc            EventName  \
0  2021-09-05 08:04:08 UTC  DynamicPrice_Result   
1  2021-08-18 11:43:23 UTC  DynamicPrice_Result   
2  2021-09-05 09:04:04 UTC  DynamicPrice_Result   
3  2021-08-25 05:02:55 UTC   CurateOffer_Result   
4  2021-09-05 08:03:28 UTC  DynamicPrice_Result   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

### Observations

* all values are present, no missing data
* EnqueuedTimeUtc is an object, not a datetime
    * this is a TODO item
* virtually all information is in the payload col
* Most of the records are EventName: DynamicPrice_Results


In [3]:
#############
# Sec1
#############
df['EnqueuedTimeUtc'] = pd.to_datetime(df['EnqueuedTimeUtc']).dt.tz_convert(tz="brazil/East")
#verify
print(df['EnqueuedTimeUtc'].head(), "\n\n\n", df.dtypes, sep="")

0   2021-09-05 05:04:08-03:00
1   2021-08-18 08:43:23-03:00
2   2021-09-05 06:04:04-03:00
3   2021-08-25 02:02:55-03:00
4   2021-09-05 05:03:28-03:00
Name: EnqueuedTimeUtc, dtype: datetime64[ns, Brazil/East]


EnqueuedTimeUtc    datetime64[ns, Brazil/East]
EventName                               object
Payload                                 object
dtype: object


### Observations

* EnqueuedTimeUtc is now datetime

## DynamicPrice_Result

Since DynamicPrice_Result comprises the majority of the records lets start there.

In [4]:
#############
# DynamicPrice_Result
#############

df2 = df.query('EventName == "DynamicPrice_Result"')
print("df2\n", df2.head(5))

def dynamic_price_option(row):
    """applied to each row where the EventName is DynamicPrice_Result, 
    with provider ApplyDynamicPricePerOption, 
    return the unpacked rows as a list of dicts"""
    # structure the dt to required format
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    if payload_row["provider"] == "ApplyDynamicPricePerOption":
        # grab top level info shared by each dict
        provider = f"\"{payload_row['provider']}\""
        offer_id = f"\"{payload_row['offerId']}\""
        for unique_option in payload_row["algorithmOutput"]:
            # open each dict and grab info,
            # then assemble everything to create new record as a dict
            # and add to result
            price_option = {
                "Provider": provider,
                "OfferId": offer_id,
                "UniqueOptionId": f"\"{unique_option['uniqueOptionId']}\"",
                "BestPrice": unique_option["bestPrice"],
                "EnqueuedTimeSP": enqueued_time
            }
            result.append(price_option)
    return result

#apply the dynamic_price_option to df
lst = df2.apply(dynamic_price_option, axis=1)
#generate master list of dicts
flatten = [x for xs in lst for x in xs]
#create df
dynamic_price_option_payload = pd.DataFrame(flatten)

print("\n\nProperly Unpacked DynamicPrice_Result:\n", dynamic_price_option_payload)

df2
             EnqueuedTimeUtc            EventName  \
0 2021-09-05 05:04:08-03:00  DynamicPrice_Result   
1 2021-08-18 08:43:23-03:00  DynamicPrice_Result   
2 2021-09-05 06:04:04-03:00  DynamicPrice_Result   
4 2021-09-05 05:03:28-03:00  DynamicPrice_Result   
5 2021-08-18 08:44:00-03:00  DynamicPrice_Result   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

### Observation

* Visual inspection: looks good.
* lets save to CSV as required by prompt
    * _uncomment the cell below to create the CSV_

In [5]:
#pd.DataFrame.to_csv('dynamic_price_option_payload.csv')

## DynamicPriceRange

The third requested CSV is also Of EventName DynamicPrice_Result so we'll work on that next, reusing the structure of the code from the previous section.

In [6]:
#############
# DynamicPriceRange
#############

def dynamic_price_range(row):
    """applied to each row where the EventName is DynamicPrice_Result, 
    with provider ApplyDynamicPriceRange, 
    return the unpacked rows as a list of dicts"""
    # structure the dt to required format
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    if payload_row["provider"] == "ApplyDynamicPriceRange":
        algorithmOutput = payload_row['algorithmOutput']
        price_range = {
            'provider': f"\"{payload_row['provider']}\"",
            'offerId': f"\"{payload_row['offerId']}\"",
            'min_global': algorithmOutput['min_global'],
            'min_recommended': algorithmOutput['min_recommended'],
            'max_recommended': algorithmOutput['max_recommended'],
            'differenceMinRecommendMinTheory': algorithmOutput['differenceMinRecommendMinTheory'],
            'EnqueuedTimeSP': enqueued_time
        }
        result.append(price_range)
    return result

#apply the dynamic_price_option to df
lst = df2.apply(dynamic_price_range, axis=1)
#generate master list of dicts
flatten = [x for xs in lst for x in xs]
#create df
DynamicPriceRange = pd.DataFrame(flatten)

print(DynamicPriceRange)

                    provider                                 offerId  \
0   "ApplyDynamicPriceRange"  "a6611d55-9624-4381-8cdd-323ee3689241"   
1   "ApplyDynamicPriceRange"  "b8c636fa-8241-47dc-ac40-bdf438a04d9c"   
2   "ApplyDynamicPriceRange"  "3d32f7fb-396d-4d3f-b673-dea1f7dc41b7"   
3   "ApplyDynamicPriceRange"  "329194f3-95a4-45ef-b3d0-2796f74ce2a0"   
4   "ApplyDynamicPriceRange"  "fdcfde5c-113d-4a59-9ae0-8bc31e2943d8"   
5   "ApplyDynamicPriceRange"  "27bbc4fa-2388-4780-b66c-92a51397d191"   
6   "ApplyDynamicPriceRange"  "baffc30b-7642-45fe-a2ce-da31a71732ae"   
7   "ApplyDynamicPriceRange"  "b5982abd-f602-47ac-b45a-bb43bf993d46"   
8   "ApplyDynamicPriceRange"  "f6643886-4a0f-45ae-ae32-ee95c72ee94a"   
9   "ApplyDynamicPriceRange"  "16a2d492-b1c3-40ec-970f-b8704d8db96f"   
10  "ApplyDynamicPriceRange"  "e94cf567-d77a-4874-9663-0b49d19f604c"   
11  "ApplyDynamicPriceRange"  "57a52d66-93fb-40a4-b44a-662d439ce070"   
12  "ApplyDynamicPriceRange"  "0474ba06-1a1e-4ebe-934f-b416dd0c9

### Observation

* Visual inspection: looks good.
* lets save to CSV as required by prompt
    * _uncomment the cell below to create the CSV_

In [7]:
#pd.DataFrame.to_csv('DynamicPriceRange.csv')

## CurateOffer_Result

Finally the last requested csv.

In [8]:
#############
# CurateOffer_Result
#############

df3 = df.query('EventName == "CurateOffer_Result"')
print(df3.head(1))

def CurrateOffer_Result(row):
    """Transform the JSON payload into a dynamic price option dataframe.
    Output for each row will be a list of dicts"""
    # structure the dt to required format
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    for ele in payload_row:
        curationProvider = f"\"{ele['curationProvider']}\""
        offerId = f"\"{ele['offerId']}\""
        dealerId = f"\"{ele['dealerId']}\""
        for option in ele['options']:
            offer_option = {
                'curationProvider' : curationProvider,
                'offerId': offerId,
                'dealerId': dealerId,
                "UniqueOptionId": f"\"{option['uniqueOptionId']}\"",
                'optionId': f"\"{option['optionId']}\"",
                'isMobileDealer': option['isMobileDealer'],
                'isOpen': option['isOpen'],
                'eta': f"\"{option['eta']}\"",
                'chamaScore': option['chamaScore'],
                'productBrand': f"\"{option['productBrand']}\"",
                'isWinner': option['isWinner'],
                'minimumPrice': option['minimumPrice'],
                'maximumPrice': option['maximumPrice'],
                'dynamicPrice': option['dynamicPrice'],
                'finalPrice': option['finalPrice'],
                "EnqueuedTimeSP": enqueued_time
            }
            if offer_option['isWinner'] == False:
                offer_option['defeatPrimaryReason'] = f"\"{option['defeatPrimaryReason']}\""
                offer_option['defeatReasons'] = f"\"{option['defeatReasons']}\""
            else:
                offer_option['defeatPrimaryReason'] = "NA"
                offer_option['defeatReasons'] = "NA"
            result.append(offer_option)
    return result

#apply the dynamic_price_option to df
lst = df3.apply(CurrateOffer_Result, axis=1)
#generate master list of dicts
flatten = [x for xs in lst for x in xs]
#create df
CurratedOffer_payload = pd.DataFrame(flatten)

print("\n\nCuratedOffer\n", CurratedOffer_payload, sep="")


            EnqueuedTimeUtc           EventName  \
3 2021-08-25 02:02:55-03:00  CurateOffer_Result   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

### Observation

* Visual inspection: looks good.
    * important to check that where IsWinner == False that the dict thereafter with defeatreasons is detailed
    * where IsWInner == True we decided to fill in in the defeated cols with NA
* lets save to CSV as required by prompt
    * _uncomment the cell below to create the CSV_

In [9]:
#pd.DataFrame.to_csv('CuratedOffer.csv')